# 異なる長さの文字列の整理（Organizing Strings of Different Lengths）

## 背景

実際の辞書では「APPLE」が「APPLET」の前に並ぶように、長さの異なる文字列を順序づけることができる。

長さが $m$ の文字列 $s$ と、長さが $n$ の文字列 $t$（$m < n$）を比較するとき、順序の決め方は次のようになる。

1. **$s$ が $t$ の接頭辞（prefix）と一致する場合**  
　例：$s$=APPLE, $t$=APPLET のとき、$s$ は $t$ より小さい（$s <_{Lex} t$）。

1. **$s$ が $t$ の接頭辞と一致しない場合**  
　例：$s$=APPLET, $t$=ARTS のとき、$s$ の先頭から $m$ 文字と $t$ の先頭 $m$ 文字（$t'$）を比較し、辞書順で小さい方が先になる。  
　この場合、APPL < ARTS なので APPLET < ARTS となる。

## 問題（入力と出力）

### 入力

1. **順序が定義されたアルファベット $A$**  
　最大12個のシンボルからなる順列（例：`D N A`）。この順序が辞書順の基準となる。

2. **正の整数 $n$**  
　生成する文字列の最大長（$n \le 4$）。

### 出力

アルファベット $A$ を用いて作成できる**長さ $n$ 以下**のすべての文字列を、上記の辞書式順序ルールに従って並べ、リストとして出力する。


### サンプル

```text
D N A
3
```

### 出力例

```
D
DD
DDD
DDN
DDA
DN
DND
DNN
DNA
DA
DAD
DAN
DAA
N
ND
NDD
NDN
NDA
NN
NND
NNN
NNA
NA
NAD
NAN
NAA
A
AD
ADD
ADN
ADA
AN
AND
ANN
ANA
AA
AAD
AAN
AAA
```

# 解法


## 考え方

比較ルールは
- 接頭辞が一致なら短い方が先
- 最初に違う文字で大小が決まる

```text
            根
             |
         ┌──┬──┬──┐
         D   N   A         
        /|\  /|\  /|\
       D N A D N A D N A ① D → DD ③ DDA → DN
      /|\
     D N A               ② DD → DDD → DDN → DDA

```

 のように、**深さ優先探索（DFS）**で文字列を生成すれば、辞書式順序になる。

In [3]:
def generate_lexicographically_stack(alphabet: list[str], n: int) -> list[str]:
    """
    スタックを使用して、辞書式順序で文字列を生成するDFS（深さ優先探索）。

    Args:
        alphabet (list): 順序付けられたアルファベットのリスト (例: ["D", "N", "A"])
        n (int): 生成する文字列の最大長 (例: 3)

    Returns:
        list: 辞書式順序でソートされた文字列のリスト
    """
    results = []

    stack = []

    # 1. 探索の起点となる文字をスタックに積む
    #    辞書順 (D, N, A) の先頭から処理するため、
    #    アルファベットの *逆順* (A, N, D) でスタックに積む
    reversed_alphabet = alphabet[::-1]
    stack = [char for char in reversed_alphabet]

    # 2. スタックが空になるまで探索を続ける
    while stack:
        # 3. スタックから現在の文字列を取り出す (LIFO)
        #    (例: 最初に "D" が取り出される)
        current_string = stack.pop()

        # 4. 取り出した文字列を結果リストに追加
        results.append(current_string)

        # 5. 現在の文字列の長さが n 未満の場合、
        #    さらに文字を1つ追加した新しい文字列をスタックに積む
        if len(current_string) < n:
            # 6. ここでも *逆順* (A, N, D) で積む
            #    (例: current_string="D" の場合、"DA", "DN", "DD" の順で積む)
            for char in reversed_alphabet:
                next_string = current_string + char
                stack.append(next_string)
                # これにより、次に pop() されるのは "DD" となる

    return results

alphabet_list = ["D", "N", "A"]
n = 3

output = generate_lexicographically_stack(alphabet_list, n)

# 結果の表示
for s in output:
    print(s)

D
DD
DDD
DDN
DDA
DN
DND
DNN
DNA
DA
DAD
DAN
DAA
N
ND
NDD
NDN
NDA
NN
NND
NNN
NNA
NA
NAD
NAN
NAA
A
AD
ADD
ADN
ADA
AN
AND
ANN
ANA
AA
AAD
AAN
AAA


## 深さ優先探索（DFS）

**スタック（LIFO: 後入れ先出しのデータ構造）** を利用してDFS（深さ優先探索）を実行し、辞書式順序で文字列を生成します。

> [!NOTE]
> 再帰を使わないのは、私が再帰が苦手だからです。
> あと、Pythonと再帰の相性が良くない（再帰深度制限があるし、そもそも遅い）ためです。

`alphabet = ["D", "N", "A"]` と `n = 3` を例として、処理の流れを具体的に説明します。


---

### 「逆順」に積む理由

スタックは「最後に入れたもの（Last-In）を最初に出す（First-Out）」構造 (LIFO) です。

* 辞書順では `"D"` を `"N"` や `"A"` より先に処理したいです。
* スタックから `"D"` を最初に取り出す（`pop`）ためには、スタックに積む（`push`/`append`）ときに**アルファベットの逆順**（`"A"` → `"N"` → `"D"`）で積む必要があります。

これにより、スタックの一番上（最後に入れられたもの）が、常に「次に辞書順で処理すべき文字列」になります。

---

### ステップ1：初期化と起点の投入

1.  `alphabet` は `["D", "N", "A"]` です。
2.  `results = []` （空のリスト）
3.  `stack = []` （空のスタック）
4.  `reversed_alphabet` は `["A", "N", "D"]` になります。
5.  **`for char in reversed_alphabet:` ループ**
    * `stack.append("A")`
    * `stack.append("N")`
    * `stack.append("D")`
6.  **初期状態完了:**
    * `results = []`
    * `stack = ["A", "N", "D"]` （底が "A", 一番上が "D"）

---

### 🔄 ステップ2：`while stack:` ループ（"D"系統の処理）

`while` ループは、スタックが空になるまで続きます。

**1周目:**
* スタック `["A", "N", "D"]` から `pop()` します。
* `current_string = "D"`
* `results.append("D")` します。
    * `results = ["D"]`
* `len("D")` (1) は `n` (3) 未満です。
* **`for char in reversed_alphabet:` ループ**
    * `next_string = "D" + "A"` -> `"DA"`。 `stack.append("DA")`
    * `next_string = "D" + "N"` -> `"DN"`。 `stack.append("DN")`
    * `next_string = "D" + "D"` -> `"DD"`。 `stack.append("DD")`
* **1周目終了時の状態:**
    * `results = ["D"]`
    * `stack = ["A", "N", "DA", "DN", "DD"]` （一番上が "DD"）

**2周目:**
* スタックから `pop()` します。
* `current_string = "DD"`
* `results.append("DD")` します。
    * `results = ["D", "DD"]`
* `len("DD")` (2) は `n` (3) 未満です。
* **`for` ループ**
    * `stack.append("DDA")`
    * `stack.append("DDN")`
    * `stack.append("DDD")`
* **2周目終了時の状態:**
    * `results = ["D", "DD"]`
    * `stack = ["A", "N", "DA", "DN", "DDA", "DDN", "DDD"]` （一番上が "DDD"）

**3周目:**
* スタックから `pop()` します。
* `current_string = "DDD"`
* `results.append("DDD")` します。
    * `results = ["D", "DD", "DDD"]`
* `len("DDD")` (3) は `n` (3) 未満では**ない**ため、ifブロックは実行されません。
* **3周目終了時の状態:**
    * `results = ["D", "DD", "DDD"]`
    * `stack = ["A", "N", "DA", "DN", "DDA", "DDN"]` （一番上が "DDN"）

**4周目:**
* スタックから `pop()` します。
* `current_string = "DDN"`
* `results.append("DDN")` します。
    * `results = ["D", "DD", "DDD", "DDN"]`
* `len("DDN")` (3) は `n` 未満では**ない**ため、ifブロックは実行されません。

**5周目:**
* スタックから `pop()` します。
* `current_string = "DDA"`
* `results.append("DDA")` します。
    * `results = ["D", "DD", "DDD", "DDN", "DDA"]`
* `len("DDA")` (3) は `n` 未満では**ない**ため、ifブロックは実行されません。

**6周目:**
* スタックから `pop()` します。
* `current_string = "DN"`
* `results.append("DN")` します。
    * `results = [..., "DDA", "DN"]`
* `len("DN")` (2) は `n` (3) 未満です。
* **`for` ループ（コメント `6.`）**
    * `stack.append("DNA")`
    * `stack.append("DNN")`
    * `stack.append("DND")`
* **6周目終了時の状態:**
    * `results = [..., "DDA", "DN"]`
    * `stack = ["A", "N", "DA", "DNA", "DNN", "DND"]` （一番上が "DND"）

**(中略)...**

このプロセス（`DND`, `DNN`, `DNA` を `pop`）が続きます。
次に `"DA"` が `pop` され、`"DAA"`, `"DAN"`, `"DAD"` がスタックに積まれます。
`"D"` で始まるものがすべて処理されると、スタックの一番上は `"N"` になります。

---

### 🔄 ステップ3：`while stack:` ループ（"N"系統の処理）

**N周目:**
* `D`系統がすべてなくなり、スタックが `["A", "N"]` になったとします。
* スタックから `pop()` します。
* `current_string = "N"`
* `results.append("N")` します。
    * `results = [..., "DAA", "N"]`
* `len("N")` (1) は `n` (3) 未満です。
* **`for` ループ（コメント `6.`）**
    * `stack.append("NA")`
    * `stack.append("NN")`
    * `stack.append("ND")`
* **N周目終了時の状態:**
    * `results = [..., "N"]`
    * `stack = ["A", "NA", "NN", "ND"]` （一番上が "ND"）

この後は、`"D"` 系統の時と同様に、`"ND"` が `pop` され、次に `"NDA"`, `"NDN"`, `"NDD"` が積まれ... という処理が `N` 系統で繰り返されます。

---

### 🔄 ステップ4：`while stack:` ループ（"A"系統の処理）

`N` 系統の処理がすべて終わると、スタックには `"A"` だけが残ります。
`"A"` が `pop` され、`"AA"`, `"AN"`, `"AD"` が積まれ、同様のプロセスが繰り返されます。

---

### 🔚 ステップ5：終了

`A` 系統の最後の文字列（`"AAA"`）が `pop` され、`results` に追加されます。
`len("AAA")` は3なので、スタックには何も積まれません。

この時点でスタックは `[]` (空) になります。
`while stack:` の条件が `False` となり、ループが終了します。

最終的に、`"D"` から `"AAA"` まで辞書式順序で格納された `results` リストが返されます。